# ONNX Inference.ipynb

## Setting Up Necessary Things

In [ ]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [ ]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# NVIDIA Status
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TC.Video.Game

/content/drive/MyDrive/TC.Video.Game


## Necessary Imports

In [ ]:
! pip install -q onnxruntime onnx onnxruntime-gpu onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
! pip install -q transformers sentencepiece ohmeow-blurr nbdev fastai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 927.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

from tqdm.notebook import tqdm
import numpy as np

## Data Cleaning

In [ ]:
# Load Data Into DataFrame
df = pd.read_csv("data/raw_data/game_details_kaggle.csv")
df.head()

,Name,Summary,Genres
0,Galactic Bowling,"Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor, wild characters, unique levels, and addictive game play. The title is aimed at players of all ages and skill sets. Through accessible and intuitive controls and game-play, Galactic Bowling allows you to jump right into the action. A single-player campaign and online play allow you to work your way up the ranks of the Galactic Bowling League! Whether you h...","['Casual', 'Indie', 'Sports']"
1,Train Bandit,"THE LAW!! Looks to be a showdown atop a train. This will be your last fight. Good luck, Train Bandit. WHAT IS THIS GAME? Train Bandit is a simple score attack game. The Law will attack you from both sides. Your weapon is your keyboard. You'll use those keys to kick the living shit out of the law. React quickly by attacking the correct direction. React...or you're dead. THE FEATURES Unlock new bandits Earn Achievements Become Steam's Most Wanted ? Battle elite officers Kick the law's ass","['Action', 'Indie']"
2,Jolt Project,"Jolt Project: The army now has a new robotics project, jolt. It's up to you to control it and ensure the success of the missions! There are 9 stages of taking the breath away with the right difficulty and good gameplay. Plus an insane way of survival! Fire missiles at cars, tanks, helicopters and turrets! The fun is guaranteed! Use your mouse to aim and shoot and take out the enemies! In this game you will have to be aware of the various enemies who will do everything to destroy your charges and prevent the success of your mission! Cartoon-style graphics are optimized and fun and generate ...","['Action', 'Adventure', 'Indie', 'Strategy']"
3,Henosis™,"HENOSIS™ is a mysterious 2D Platform Puzzler where players are propelled into weird and visceral worlds as they take control of a small, droplet of water while overcoming obstacles and enemies throughout each level. The Player must venture through each world as it collects precious water tokens in order open the exit portal and restore vitality to its drought-ridden home world. Features: Traverse your way through 27 hand-crafted levels Unique player mechanics Battle menacing bosses across 3 distinct worlds* Original artwork &amp; animation Full controller support Localization support * Hid...","['Adventure', 'Casual', 'Indie']"
4,Two Weeks in Painland,"ABOUT THE GAME Play as a hacker who has arranged a deal with a gangster. That’s how the protagonist, Jack, is assigned a mission that should be accomplished in a specific timeframe, which he will find out soon enough. THE GAME’S FEATURES Spy on 4 senior managers within an organization to find out about their personalities. Manage the recruitment process in the organization to improve the work climate. Hack the candidates who want to get into the organization to make your job easier. Try to avoid having your physical health impacted negatively in the process. All of this while you enjoy an ...","['Adventure', 'Indie']"


In [ ]:
# Shape of the DataFrame
df.shape

(76987, 3)

In [ ]:
genres_list = df["Genres"].to_list()    # list of all the genres
genres_count = {}                       # count of all the genres

for genres in genres_list:
  try:
    genre_list = eval(genres)

    for genre in genre_list:
      if genre in genres_count.keys():
        genres_count[genre] += 1
      else:
        genres_count[genre] = 1
  except:
    continue

In [ ]:
print("Total Number of Genres: ", len(genres_count))

sorted_genres_count = dict(sorted(genres_count.items(), key=lambda item: item[1], reverse=True))
print("Individual Genres Count: \n", sorted_genres_count)

Total Number of Genres:  33
Individual Genres Count: 
 {'Indie': 51987, 'Casual': 30970, 'Action': 30802, 'Adventure': 28543, 'Simulation': 14602, 'Strategy': 14114, 'RPG': 12966, 'Early Access': 9179, 'Free to Play': 6150, 'Sports': 3402, 'Racing': 2758, 'Massively Multiplayer': 1941, 'Utilities': 781, 'Violent': 509, 'Design & Illustration': 464, 'Animation & Modeling': 380, 'Education': 377, 'Gore': 305, 'Video Production': 269, 'Game Development': 208, 'Software Training': 192, 'Audio Production': 188, 'Photo Editing': 125, 'Nudity': 117, 'Sexual Content': 106, 'Web Publishing': 105, 'Accounting': 22, 'Movie': 2, 'Documentary': 1, 'Episodic': 1, 'Short': 1, 'Tutorial': 1, '360 Video': 1}


### Removing Rare Genres

In [ ]:
threshold = int(len(df) * 0.0025)

rare_genres = [item for item, count in genres_count.items() if count < threshold]

print("Total Rare Genres: ", len(rare_genres))
print("Rare Genres: ", rare_genres)

Total Rare Genres:  12
Rare Genres:  ['Sexual Content', 'Nudity', 'Web Publishing', 'Photo Editing', 'Audio Production', 'Accounting', 'Movie', 'Documentary', 'Episodic', 'Short', 'Tutorial', '360 Video']


In [ ]:
genres_list = df["Genres"].to_list()
revised_genre_list = []
indices_to_drop = []

for idx, genres in enumerate(genres_list):
  try:
    genre_list = eval(genres)
    revised_genres = []

    for genre in genre_list:
      if genre not in rare_genres:
        revised_genres.append(genre)

    if len(revised_genres) == 0:
      indices_to_drop.append(idx)
    else:
      revised_genres_set = set(revised_genres)
      revised_genre_list.append(list(revised_genres_set))
  except:
    indices_to_drop.append(idx)

In [ ]:
df = df.drop(indices_to_drop).reset_index(drop=True)
df.shape

(74001, 3)

In [ ]:
df["Revised Genres"] = revised_genre_list

In [ ]:
revised_genres_list = df["Revised Genres"].to_list()
revised_genre_count = {}

for genres in revised_genres_list:
  genre_list = genres

  for genre in genre_list:
    if genre in revised_genre_count.keys():
      revised_genre_count[genre] += 1
    else:
      revised_genre_count[genre] = 1

In [ ]:
print("Total Number of Revised Genres: ", len(revised_genre_count))

sorted_revised_genre_count = dict(sorted(revised_genre_count.items(), key=lambda item: item[1], reverse=True))
print("Individual Revised Genres Count: \n", sorted_revised_genre_count)

Total Number of Revised Genres:  21
Individual Revised Genres Count: 
 {'Indie': 51987, 'Casual': 30970, 'Action': 30802, 'Adventure': 28543, 'Simulation': 14602, 'Strategy': 14114, 'RPG': 12966, 'Early Access': 9179, 'Free to Play': 6150, 'Sports': 3402, 'Racing': 2758, 'Massively Multiplayer': 1941, 'Utilities': 781, 'Violent': 509, 'Design & Illustration': 464, 'Animation & Modeling': 380, 'Education': 377, 'Gore': 305, 'Video Production': 269, 'Game Development': 208, 'Software Training': 192}


In [ ]:
encode_genre_types = { key: idx for idx, (key, value) in enumerate(revised_genre_count.items())}

with open("genre_types_encoded_kaggle_onnx.json", "w") as fp:
  json.dump(encode_genre_types, fp)

In [ ]:
encode_genre_types

{'Casual': 0,
 'Indie': 1,
 'Sports': 2,
 'Action': 3,
 'Adventure': 4,
 'Strategy': 5,
 'Massively Multiplayer': 6,
 'RPG': 7,
 'Free to Play': 8,
 'Simulation': 9,
 'Early Access': 10,
 'Racing': 11,
 'Utilities': 12,
 'Education': 13,
 'Violent': 14,
 'Gore': 15,
 'Design & Illustration': 16,
 'Animation & Modeling': 17,
 'Game Development': 18,
 'Software Training': 19,
 'Video Production': 20}

In [ ]:
# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_genre_list = []
revised_genres_list = df["Revised Genres"].to_list()

for revised_genres in revised_genres_list:
  categorical_list = [0] * len(encode_genre_types)
  genre_list = revised_genres

  for genre in genre_list:
    if genre in encode_genre_types:
      genre_type_index = encode_genre_types[genre]
      categorical_list[genre_type_index] = 1

  categorical_genre_list.append(categorical_list)

categorical_genre_list[3][:23]

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
df["Genre Cat List"] = categorical_genre_list

In [ ]:
df.head()

,Name,Summary,Genres,Revised Genres,Genre Cat List
0,Galactic Bowling,"Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor, wild characters, unique levels, and addictive game play. The title is aimed at players of all ages and skill sets. Through accessible and intuitive controls and game-play, Galactic Bowling allows you to jump right into the action. A single-player campaign and online play allow you to work your way up the ranks of the Galactic Bowling League! Whether you h...","['Casual', 'Indie', 'Sports']","[Casual, Indie, Sports]","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Train Bandit,"THE LAW!! Looks to be a showdown atop a train. This will be your last fight. Good luck, Train Bandit. WHAT IS THIS GAME? Train Bandit is a simple score attack game. The Law will attack you from both sides. Your weapon is your keyboard. You'll use those keys to kick the living shit out of the law. React quickly by attacking the correct direction. React...or you're dead. THE FEATURES Unlock new bandits Earn Achievements Become Steam's Most Wanted ? Battle elite officers Kick the law's ass","['Action', 'Indie']","[Action, Indie]","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Jolt Project,"Jolt Project: The army now has a new robotics project, jolt. It's up to you to control it and ensure the success of the missions! There are 9 stages of taking the breath away with the right difficulty and good gameplay. Plus an insane way of survival! Fire missiles at cars, tanks, helicopters and turrets! The fun is guaranteed! Use your mouse to aim and shoot and take out the enemies! In this game you will have to be aware of the various enemies who will do everything to destroy your charges and prevent the success of your mission! Cartoon-style graphics are optimized and fun and generate ...","['Action', 'Adventure', 'Indie', 'Strategy']","[Adventure, Action, Indie, Strategy]","[0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Henosis™,"HENOSIS™ is a mysterious 2D Platform Puzzler where players are propelled into weird and visceral worlds as they take control of a small, droplet of water while overcoming obstacles and enemies throughout each level. The Player must venture through each world as it collects precious water tokens in order open the exit portal and restore vitality to its drought-ridden home world. Features: Traverse your way through 27 hand-crafted levels Unique player mechanics Battle menacing bosses across 3 distinct worlds* Original artwork &amp; animation Full controller support Localization support * Hid...","['Adventure', 'Casual', 'Indie']","[Adventure, Indie, Casual]","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,Two Weeks in Painland,"ABOUT THE GAME Play as a hacker who has arranged a deal with a gangster. That’s how the protagonist, Jack, is assigned a mission that should be accomplished in a specific timeframe, which he will find out soon enough. THE GAME’S FEATURES Spy on 4 senior managers within an organization to find out about their personalities. Manage the recruitment process in the organization to improve the work climate. Hack the candidates who want to get into the organization to make your job easier. Try to avoid having your physical health impacted negatively in the process. All of this while you enjoy an ...","['Adventure', 'Indie']","[Adventure, Indie]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# Drop Duplicate Summary from the DataFrame
df = df.drop_duplicates(subset = "Summary", keep='first')

In [ ]:
# Drop Genres Column
df.drop(["Genres"], axis = 1, inplace = True)

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.isna().sum()

Name              0
Summary           0
Revised Genres    0
Genre Cat List    0
dtype: int64

In [ ]:
# Description of The DataFrame
df.describe(include="object")

,Name,Summary,Revised Genres,Genre Cat List
count,73551,73551,73551,73551
unique,73057,73551,2026,2016
top,Escape,"Galactic Bowling is an exaggerated and stylized bowling game with an intergalactic twist. Players will engage in fast-paced single and multi-player competition while being submerged in a unique new universe filled with over-the-top humor, wild characters, unique levels, and addictive game play. The title is aimed at players of all ages and skill sets. Through accessible and intuitive controls and game-play, Galactic Bowling allows you to jump right into the action. A single-player campaign and online play allow you to work your way up the ranks of the Galactic Bowling League! Whether you h...","[Casual, Indie]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
freq,5,1,4300,4300


In [ ]:
labels = list(encode_genre_types.keys())

print("Total Labels: ", len(labels))
print("Labels: ", labels)

Total Labels:  21
Labels:  ['Casual', 'Indie', 'Sports', 'Action', 'Adventure', 'Strategy', 'Massively Multiplayer', 'RPG', 'Free to Play', 'Simulation', 'Early Access', 'Racing', 'Utilities', 'Education', 'Violent', 'Gore', 'Design & Illustration', 'Animation & Modeling', 'Game Development', 'Software Training', 'Video Production']


## Data Split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)

train_ids, valid_ids = splitter(df)

print("Train Len: ", len(train_ids))
print("Valid Len: ", len(valid_ids))

Train Len:  66196
Valid Len:  7355


In [ ]:
valid_df = df.loc[valid_ids]

print("Valid Data: ")
valid_df.head()

Valid Data: 


,Name,Summary,Revised Genres,Genre Cat List
3621,Fortoresse,"Fortoresse is a 2D team-based pvp warfare game in the simplest but most efficient fashion! Shoot and conjure blocks at your advantage to lead your team to victory! Cooperation is key! Work with your teammates! While one is aiming for the target, the other can secure the way by conjuring cover walls or platforms ! Those act differently depending on who summons them! Multiple game modes! Choose between VIP vs VIP , where you have to protect the VIP of your team while hunting for the enemy’s; Capture the flag while making sure yours is safe; or simply hop on a standard Death match Domination!","[Strategy, Action, Free to Play]","[0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
21368,Xyla and the 'Shrooms of Doom,Xyla is a pink haired hero who was raised as an elite Forest Agent. She realised early on in her life that dedication to her jumping and shooting skills would lead her to success. Work with Xyla as she navigates the unpredictable terrain of the forest. Xyla’s unique jumping and flying techniques allow movement not only the traditional left to right movement but allows Xyla to explore maps in a in a way that uses the whole screen to it’s fullest potential.,"[Casual, Action, Indie]","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
67045,Operation: Harsh Doorstop,"What is Operation: Harsh Doorstop? Operation: Harsh Doorstop is a shooter sandbox with all the features a game should have. Single player missions, cooperative modes, multiplayer servers, and Steam workshop support. These things should be basic, and all games should have them. However, for some reason... games rarely do anymore. That is the reason we started this project. How much does Operation: Harsh Doorstop cost? Operation: Harsh Doorstop is a free game. Our community members can donate to help fund development, but we still believe in making our game freely accessible to everyone. Not...","[Simulation, Casual, Action, Strategy, Indie, Early Access, Free to Play]","[1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
241,Cub Gym,"At Cub Gym the player will have a huge challenge ahead, several crazy games are waiting for him trying to take him down, test his skills and make it to the end. With a completely random level creation system, Cub Gym promises different routes for each match, always trying to surprise the player by always offering a new experience. In this course that never seems to end, the player will have to face many tranpolins, risky jumps, objects thrown in addition to having a good sense of time and balance. Get to the end, advance on this arduous path and always be victorious, but be careful they ar...","[Casual, Action, Indie]","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
49968,Ino,"INO is a short Adventure/Horror story driven game. It tells the story of Kaede and Hitaki, two classmates that get involved with a mysterious girl called Ino. Follow the story as they try to figure out who she is while they try to escape from being murdered. Features: - Unique maps with original graphic assets - Chase system to keep you on your toes - Puzzles to test your wits - Character driven story","[Adventure, Violent, Gore, Indie, RPG]","[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]"


*We will be using `valid_df` for all inference testing*

## Fastai and Blurr Inference

In [ ]:
model_path = "models/game-ml-classifier-stage-kaggle-1.pkl"

learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("Action Adventure")

[{'labels': ['Indie', 'Action', 'Adventure'],
  'scores': [0.8056983947753906, 0.9840582013130188, 0.9737042188644409],
  'class_indices': [1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['Indie', 'Sports', 'Casual', 'Action', 'Adventure', 'Strategy', 'Free to Play', 'RPG', 'Massively Multiplayer', 'Simulation', 'Early Access', 'Racing', 'Utilities', 'Education', 'Violent', 'Gore', 'Design & Illustration', 'Animation & Modeling', 'Game Development', 'Software Training', 'Video Production'],
  'probs': [0.8056983947753906,
   0.0036832201294600964,
   0.07712947577238083,
   0.9840582013130188,
   0.9737042188644409,
   0.02780238911509514,
   0.021536167711019516,
   0.36293360590934753,
   0.006218271795660257,
   0.01700422912836075,
   0.15098552405834198,
   0.006608179770410061,
   0.00013823140761815012,
   0.00018496277334634215,
   0.018300266936421394,
   0.018372926861047745,
   0.0

In [ ]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['Indie', 'RPG']

## Evaluation

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['Genre Cat List'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Summary']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

preds[0][:20]

  0%|          | 0/7355 [00:00<?, ?it/s]

[0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.8275455572474092
F1 Score (Macro) = 0.6467502737847393


## Convert to ONNX

In [ ]:
model_path = "models/game-ml-classifier-stage-kaggle-1.pkl"
learner_inf = load_learner(model_path)

In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/game-classifier-kaggle.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/game-classifier-kaggle.onnx'
quantized_onnx_model_path = 'models/game-classifier-quantized-kaggle.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

## ONNX Inference

### Normal ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/game-classifier-kaggle.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/7355 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.583683809242726
F1 Score (Macro) = 0.5289314131273253


### Quantized ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_genre_types.keys())

inf_session = rt.InferenceSession('models/game-classifier-quantized-kaggle.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_genre_types)
  for label in labels:
    pred_genres[encode_genre_types[label]] = 1
  preds.append(pred_genres)

  0%|          | 0/7355 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.5847174447174447
F1 Score (Macro) = 0.527175324052185
